In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder

In [18]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)

Using BeautifulSoup and requests to scrape the table into a pandas dataframe

In [19]:
soup = BeautifulSoup(res.content, 'html.parser')

In [20]:
table = soup.find_all('table')[0]
start = pd.read_html(str(table))

In [21]:
df = pd.DataFrame(start[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


There is only one neighbourhood that is not assigned but has an assigned borough.
Instead of just changing it, this code is designed to change all values if more values are ever added.

In [22]:
msk = (df['Neighbourhood']=='Not assigned') & (df['Borough']!='Not assigned')
df.loc[msk, 'Neighbourhood'] = df.loc[msk, 'Borough']



Now we remove all values that are not assigned

In [23]:
df = df[df['Neighbourhood']!='Not assigned']
df.shape

(210, 3)

Group all postcodes w

In [24]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [29]:
df.shape

(103, 3)

In [31]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
df_latlng = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
df_latlng.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_latlng.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
df = df.merge(df_latlng, on='Postcode')

In [43]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


,Postcode,Borough,Neighbourhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
